## ch9.2

### 원하는 숫자 생성하기

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


#### 생성자 신경망 구성
- tf.layers 를 이용
- 변수를 선언하지 않고 다음과 같이 스코프 지정 가능

In [2]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가합니다.
        inputs = tf.concat([noise, labels], 1)

        # TensorFlow 에서 제공하는 유틸리티 함수를 이용해 신경망을 매우 간단하게 구성할 수 있습니다.
        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                 activation=tf.nn.sigmoid)

    return output

#### 구분자 신경망 구성
- 진짜와 가짜를 판별할때 똑같은 변수를 사용해야 한다. 그러기위해 scope.reuse_variables 이용하여 재사용하도록 함
- 출력값은 손실값 계산에 sigmoid_cross_entropy_with_logits 함수사용

In [3]:
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용하도록 합니다.
        if reuse:
            scope.reuse_variables()

        inputs = tf.concat([inputs, labels], 1)

        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1,
                                 activation=None)

    return output


#### 노이즈 생성함수
- 균등분포로 생성

In [4]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [5]:
# 생성 모델과 판별 모델에 Y 즉, labels 정보를 추가하여
# labels 정보에 해당하는 이미지를 생성할 수 있도록 유도합니다.
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))

loss_D = loss_D_real + loss_D_gene

loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D,var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G,var_list=vars_G)

#### 학습진행

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Y: batch_ys, Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G,
                           feed_dict={Y: mnist.test.labels[:sample_size],
                                      Z: noise})

        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()

            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('ch9-2_samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: 0.005998 G loss: 7.291
Epoch: 0001 D loss: 0.01454 G loss: 8.082
Epoch: 0002 D loss: 0.01689 G loss: 8.025
Epoch: 0003 D loss: 0.0271 G loss: 6.799
Epoch: 0004 D loss: 0.01996 G loss: 7.088
Epoch: 0005 D loss: 0.0293 G loss: 7.054
Epoch: 0006 D loss: 0.09302 G loss: 7.97
Epoch: 0007 D loss: 0.1017 G loss: 6.465
Epoch: 0008 D loss: 0.0775 G loss: 6.327
Epoch: 0009 D loss: 0.1886 G loss: 5.536
Epoch: 0010 D loss: 0.4207 G loss: 4.667
Epoch: 0011 D loss: 0.3436 G loss: 3.867
Epoch: 0012 D loss: 0.1963 G loss: 5.271
Epoch: 0013 D loss: 0.3338 G loss: 5.002
Epoch: 0014 D loss: 0.3873 G loss: 4.466
Epoch: 0015 D loss: 0.5526 G loss: 4.446
Epoch: 0016 D loss: 0.4301 G loss: 3.441
Epoch: 0017 D loss: 0.5794 G loss: 3.804
Epoch: 0018 D loss: 0.4653 G loss: 3.329
Epoch: 0019 D loss: 0.5479 G loss: 3.141
Epoch: 0020 D loss: 0.8177 G loss: 2.587
Epoch: 0021 D loss: 0.6188 G loss: 3.215
Epoch: 0022 D loss: 0.6106 G loss: 2.889
Epoch: 0023 D loss: 0.7095 G loss: 2.931
Epoch: 0024